## Sobre o dataset
O conjunto de dados veio de uma enquete com os clientes da BoraBusão e queremos saber se com estes dados podemos prever a satisfação dos mesmos com os serviços da empresa. (Lembrando que tanto a empresa citada quanto os dados são fictícios e alterados)

### Features e contexto
* ID: Identificação do cliente
* Genero: Gênero do cliente
* PlanoFidelidade: Se o cliente possui ou não o plano fidelidade da BoraBusão
* Idade: Idade do cliente
* RazaoViagem: Motivo da viagem ( pessoal ou a trabalho? )
* CategoriaPassagem: Em qual catergoria ele está viajando? Normal, Comforto ou Leito
* DistanciaKm: A distancia do trecho de viagem
* WiFi: Possui WiFi no ônibus, o serviço está bom?
* ConvenienciaHorarios: Os horários de partida e chagada são convenientes?
* FacilidadeReservaViaApp: Nível de facilidade de fazer a reserva da passagem
* PontosLocalização: A localização dos pontos de ônibus é boa, qual a satisfação com relação a esse ponto
* Alimentação: A alimentação servida no oninbus e nos pontos, qual a sua avaliação?
* CheckInViaApp: Facilidade de fazer o checkIn via o app
* ConfortoInterno: Nível de conforto do ônibus ( cadeiras, ar-condicionado)
* ServicosIntegracao: Nível de satisfação desde a chegada até o embarque.
* SalaDeEspera: Nível de satisfação com a sala de espera de quem tem o plano Fidelidade
* Bagagem: Nível de satisfação com o serviço e manuseamento da bagagem do passageiro
* ServicoCheckin: Nivel de satisfaçao com o serviço de checkin local
* ServicoDeBordo: Nível de satisfação com o serviço de bordo
* Limpeza: Nível de satisfação com a Limpeza
* AtrasoNaSaída: Atraso em minutos na partida
* AtrasoNaChegada: Atraso em minuto na chegada
* SatisfacaoGeral: Variável alvo, o cliente está satisfeito ou não

## Importações de módulos e configurações

In [65]:
%matplotlib inline

import joblib
import numpy as np
import pandas as pd
pd.set_option('display.max_columns', None)

import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib.colors import LinearSegmentedColormap
import matplotlib.font_manager
import plotly.express as px

import mlflow

from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MinMaxScaler, StandardScaler, RobustScaler
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline, FunctionTransformer
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import Normalizer
from sklearn.base import BaseEstimator, TransformerMixin

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score, roc_curve

## Funções auxiliares

In [66]:
def faixa_idade(x, minimum_age, first_quantile, second_quantile, third_quantile, max_age):
    if(x < first_quantile):
        return f"{minimum_age}-{first_quantile}"
    elif(x < second_quantile):
        return f"{first_quantile}-{second_quantile}"
    elif(x < third_quantile):
        return f"{second_quantile}-{third_quantile}"
    else:
        return f"{third_quantile}-{max_age}"

In [67]:
def dm_colors(n_colors=10, as_cmap=False):
    if(as_cmap):
        return LinearSegmentedColormap.from_list(
        "Custom", ["#ff5871", "#68e699"], N=n_colors)
    colors = ["#202ad0", "#ff9a98", "#ffe372", "#7df4ed", "#68e699", "#000033", "#42d6fd", "#ff5871", "#ffc000", "#00c8ba"]
    return sns.color_palette(palette=colors, n_colors=n_colors, as_cmap=as_cmap)


In [68]:
def set_dm_theme():
    sns.set_theme(palette=dm_colors(), font_scale=1, font='Arial')

In [69]:
set_dm_theme()

## 1 - Data Collection

### Lendo csv

In [70]:
df = pd.read_csv('../data/raw/BoraBusTratado.csv')
df.shape

(103904, 24)

In [71]:
df.head()

,ID,Genero,PlanoFidelidade,Idade,RazaoViagem,CategoriaPassagem,DistanciaKm,WiFi,ConvenienciaHorarios,FacilidadeReservaViaApp,PontosLocalizacao,Alimentacao,CheckInViaApp,ConfortoInterno,Entretenimento,ServicosIntegracao,SalaDeEspera,Bagagem,ServicoCheckIn,ServicoDeBordo,Limpeza&Higiene,AtrasoNaSaida,AtrasoNaChegada,SatisfacaoGeral
0,70172,Masculino,Sim,13,TurismoOuPessoal,Comforto,288.0,3,4,3,1,5,3,5,5,4,3,4,4,5,5,25,18.0,Nao
1,5047,Masculino,Nao,25,NegociosOuTrabalho,Leito,147.0,3,2,3,3,1,3,1,1,1,5,3,1,4,1,1,6.0,Nao
2,110028,Feminino,Sim,26,NegociosOuTrabalho,Leito,714.0,2,2,2,2,5,5,5,5,4,3,4,4,4,5,0,0.0,Sim
3,24026,Feminino,Sim,25,NegociosOuTrabalho,Leito,351.0,2,5,5,5,2,2,2,2,2,5,3,1,4,2,11,9.0,Nao
4,119299,Masculino,Sim,61,NegociosOuTrabalho,Leito,134.0,3,3,3,3,4,5,5,3,3,4,4,3,3,3,0,0.0,Sim


In [72]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 103904 entries, 0 to 103903
Data columns (total 24 columns):
 #   Column                   Non-Null Count   Dtype  
---  ------                   --------------   -----  
 0   ID                       103904 non-null  int64  
 1   Genero                   103904 non-null  object 
 2   PlanoFidelidade          103904 non-null  object 
 3   Idade                    103904 non-null  int64  
 4   RazaoViagem              103904 non-null  object 
 5   CategoriaPassagem        103904 non-null  object 
 6   DistanciaKm              103904 non-null  float64
 7   WiFi                     103904 non-null  int64  
 8   ConvenienciaHorarios     103904 non-null  int64  
 9   FacilidadeReservaViaApp  103904 non-null  int64  
 10  PontosLocalizacao        103904 non-null  int64  
 11  Alimentacao              103904 non-null  int64  
 12  CheckInViaApp            103904 non-null  int64  
 13  ConfortoInterno          103904 non-null  int64  
 14  Entr

In [73]:
df.select_dtypes('object')

,Genero,PlanoFidelidade,RazaoViagem,CategoriaPassagem,SatisfacaoGeral
0,Masculino,Sim,TurismoOuPessoal,Comforto,Nao
1,Masculino,Nao,NegociosOuTrabalho,Leito,Nao
2,Feminino,Sim,NegociosOuTrabalho,Leito,Sim
3,Feminino,Sim,NegociosOuTrabalho,Leito,Nao
4,Masculino,Sim,NegociosOuTrabalho,Leito,Sim
...,...,...,...,...,...
103899,Feminino,Nao,NegociosOuTrabalho,Normal,Nao
103900,Masculino,Sim,NegociosOuTrabalho,Leito,Sim
103901,Masculino,Nao,NegociosOuTrabalho,Leito,Nao
103902,Feminino,Nao,NegociosOuTrabalho,Normal,Nao


## 2 - Data Preparation

### Verificação de valores inconsistentes

In [74]:
for coluna in df.columns:
    print(df[coluna].value_counts())
    print()

70172     1
116739    1
6259      1
17470     1
118574    1
         ..
107167    1
103283    1
112365    1
98359     1
62567     1
Name: ID, Length: 103904, dtype: int64

Feminino     52727
Masculino    51177
Name: Genero, dtype: int64

Sim    84923
Nao    18981
Name: PlanoFidelidade, dtype: int64

39    2969
25    2798
40    2574
44    2482
42    2457
      ... 
74      47
76      45
79      42
78      33
85      17
Name: Idade, Length: 75, dtype: int64

NegociosOuTrabalho    71655
TurismoOuPessoal      32249
Name: RazaoViagem, dtype: int64

Leito       49665
Normal      46745
Comforto     7494
Name: CategoriaPassagem, dtype: int64

211.0    797
148.0    577
231.0    497
252.0    488
279.0    485
        ... 
825.0      1
637.0      1
245.0      1
800.0      1
250.0      1
Name: DistanciaKm, Length: 2420, dtype: int64

3    25868
2    25830
4    19794
1    17840
5    11469
0     3103
Name: WiFi, dtype: int64

4    25546
5    22403
3    17966
2    17191
1    15498
0     5300
Name: Con

### Drop da coluna de ID

In [75]:
df.drop(columns=['ID'], inplace=True)

## 3 - Experimento e pipeline

### Classes para criação dos steps

#### Feature engineering

In [76]:
class FaixaIdade(BaseEstimator, TransformerMixin):
    def fit(self, X, y=None):
        return self

    # Considerando que X será um Dataframe
    def transform(self, X, y=None):
        minimum_age = df["Idade"].min()
        first_quantile = df["Idade"].quantile(0.25)
        second_quantile = df["Idade"].quantile(0.5)
        third_quantile = df["Idade"].quantile(0.75)
        maximum_age = df["Idade"].max()
        X["FaixaIdade"] = X["Idade"].apply(lambda x: faixa_idade(x, minimum_age, first_quantile, second_quantile, third_quantile, maximum_age))
        return X


In [77]:
class EncodeFeatures(BaseEstimator, TransformerMixin):
    def fit(self, X, y=None):
        return self

    # Considerando que X será um Dataframe
    def transform(self, X, y=None):
        simnao_map = {"Sim": 1, "Nao": 0}
        X["PlanoFidelidade"] = X["PlanoFidelidade"].map(simnao_map)
        X["CategoriaPassagem"] = X["CategoriaPassagem"].map({"Normal": 0, "Comforto": 1, "Leito": 2})

        for column in X.select_dtypes('object').columns:
            le = LabelEncoder()
            X[column] = le.fit_transform(X[column])

        return X


### Classes de tratamento para o pipeline

In [78]:
class SelecionaColunas(BaseEstimator, TransformerMixin):
    
    def __init__(self, columns):
        self.columns = columns
        
    def fit(self, X, y=None):
        return self
    
    def transform(self, X):
        return X[self.columns]


In [79]:
class DropNa(BaseEstimator, TransformerMixin):
    def fit(self, X, y=None):
        return self

    # Considerando que X será um Dataframe
    def transform(self, X, y=None):
        X["AtrasoNaChegada"] = X["AtrasoNaChegada"].dropna(inplace=True)
        return X


In [80]:
class FillNa(BaseEstimator, TransformerMixin):
    def fit(self, X, y=None):
        return self

    # Considerando que X será um Dataframe
    def transform(self, X, y=None):
        X["AtrasoNaChegada"] = X["AtrasoNaChegada"].fillna(0)
        return X

### Criação do experimento

In [96]:
def run_experiment(train_df, test_df, validation_df, columns, y_column, classification_model, normalizer, null_func):
    train_df[y_column] = train_df[y_column].map({"Sim": 1, "Nao": 0})
    test_df[y_column] = test_df[y_column].map({"Sim": 1, "Nao": 0})
    validation_df[y_column] = validation_df[y_column].map({"Sim": 1, "Nao": 0})

    model = Pipeline(
        steps=[
            ('seleciona_colunas', SelecionaColunas(columns)),
            ('null_drop', null_func()),
            ('encodifica_strings', EncodeFeatures()),
            ('normalizer', normalizer()),
            ('treinamento', classification_model())
        ]
    )

    mlflow.set_experiment('borabus_hackathon')
    with mlflow.start_run(run_name='BoraBus Hackathon'):

        y = train_df.pop('SatisfacaoGeral')
        X = train_df[columns]

        model.fit(X, y)

        y_test_predict = model.predict(test_df[columns])
        y_test = test_df[y_column]

        print(roc_curve(y_true=y_test, y_score=y_test_predict))



### Separação dos dados em treino e teste

In [97]:
## Realizando um shuffle nos dados
df = df.sample(df.shape[0])
df.reset_index(drop=True, inplace=True)

In [98]:
test_10 = df.iloc[0:round(len(df)*0.1)]
test_10.to_csv('../data/raw/1.0-glt-10_percent_test.csv')

In [99]:
train_df = df.iloc[round(len(df)*0.1)+1:round(len(df)*0.7)]
test_df = df.iloc[round(len(df)*0.7)+1:round(len(df)*0.85)]
validation_df = df.iloc[round(len(df)*0.85)+1:]

In [100]:
print(f'O dataframe de teste do binário tem {test_10.shape[0]/df.shape[0]*100:.2f}% dos dados')
print(f'O dataframe de treino tem {train_df.shape[0]/df.shape[0]*100:.2f}% dos dados')
print(f'O dataframe de teste tem {test_df.shape[0]/df.shape[0]*100:.2f}% dos dados')
print(f'O dataframe de validação tem {test_df.shape[0]/df.shape[0]*100:.2f}% dos dados')

O dataframe de teste do binário tem 10.00% dos dados
O dataframe de treino tem 60.00% dos dados
O dataframe de teste tem 15.00% dos dados
O dataframe de validação tem 15.00% dos dados


In [101]:
colunas = [list(df.drop(columns=['SatisfacaoGeral']).columns)]
coluna_target = 'SatisfacaoGeral'

In [102]:
models = [LogisticRegression, DecisionTreeClassifier]
normalizers = [MinMaxScaler, StandardScaler, RobustScaler]
nullfuncs = [DropNa, FillNa]

for normalizer in normalizers:
   for model in models:
       for nullfunc in nullfuncs:
           for cols in colunas:
               run_experiment(train_df.copy(), test_df.copy(), validation_df.copy(), cols, coluna_target, model, normalizer, nullfunc)
# pd.DataFrame(run_experiment(train_df, test_df, validation_df, colunas[0], coluna_target, LogisticRegression, MinMaxScaler, DropNa), columns=df.columns)

/tmp/ipykernel_53134/4084783962.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_df[y_column] = train_df[y_column].map({"Sim": 1, "Nao": 0})
/tmp/ipykernel_53134/4084783962.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_df[y_column] = test_df[y_column].map({"Sim": 1, "Nao": 0})
/tmp/ipykernel_53134/4084783962.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the docu

(array([0.        , 0.09379986, 1.        ]), array([0.        , 0.84199706, 1.        ]), array([2, 1, 0]))


KeyError: 'SatisfacaoGeral'